## Multi-Agent GroupChat
In the previous example, we show how to introduce human-in-the-loop to a two-agent chat conversation. In this example, we extend the previous example to a multi-agent chat conversation.

### Motivation
Similar to the group chat in our daily conversation, we can also introduce multiple agents into the same group where each agent can have its own personality and knowledge. This can be useful in many applications, such as customer service, where multiple agents can work together to provide better service to the customers.

### Scenario
In this notebook, we will show how to use the following agents to create a group chat conversation that hosts all the agents we created in the previous examples:
- User: ask questions
- WeatherAssistant: provides the weather information using tool call.
- ImageAssistant: provides the information of the image using multi-modal model.
- Assistant: Answer generic questions.
- Manager: orchestrates the agents.

In [1]:
from dataclasses import dataclass, field
from typing import List, Optional
import base64
from io import BytesIO
from autogen_core.base import MessageContext, AgentId, AgentInstantiationContext, TopicId
from autogen_core.components import DefaultTopicId, RoutedAgent, default_subscription, message_handler, TypeSubscription
from autogen_core.components.code_executor import CodeExecutor, extract_markdown_code_blocks
from autogen_core.components.models import (
    AssistantMessage,
    ChatCompletionClient,
    LLMMessage,
    SystemMessage,
    UserMessage,
)
import tempfile
from autogen_core.components.tool_agent import ToolAgent, tool_agent_caller_loop
from autogen_core.application import SingleThreadedAgentRuntime
from autogen_core.components.models import OpenAIChatCompletionClient
import random
import json
from autogen_core.base import CancellationToken
from autogen_core.components.tools import FunctionTool, ToolSchema
from autogen_core.components._image import Image
from typing_extensions import Annotated
from pydantic import BaseModel
from PIL import Image as PILImage
import os
import uuid;

In [2]:
class Message(BaseModel):
    text: str
    source: Optional[str] = None
    image_url: Optional[str] = None # url or base64 encoded image

class Conversation(BaseModel):
    chat_history: List[Message]

In [3]:
class UserAgent(RoutedAgent):
    def __init__(self, description: str, group_manager_topic: str) -> None:
        super().__init__(description=description)
        self._group_manager_topic = group_manager_topic

    @message_handler
    async def handle_request_to_speak(self, message: Conversation, ctx: MessageContext) -> None:
        user_input = input("Enter your message, type 'TERMINATE' to terminate the task: ")
        print(f"### User: \n{user_input}")
        if user_input == "TERMINATE": # end the conversation
            return
        
        message.chat_history.append(Message(text=user_input, source=self.type))
        await self.publish_message(
            message,
            DefaultTopicId(type=self._group_manager_topic),
        )

In [4]:
class ImageAssistant(RoutedAgent):
    def __init__(self,
                 model_client: ChatCompletionClient,
                 system_message: str,
                 group_manager_topic: str) -> None:
        super().__init__(description=system_message)
        self._system_message = system_message
        self._group_manager_topic = group_manager_topic
        self._model_client = model_client

    @message_handler
    async def handle_request_to_speak(self, message: Conversation, ctx: MessageContext) -> None:
        chat_history: List[LLMMessage] = [SystemMessage(content=self._system_message)]
        last_msg = message.chat_history[-1]
        if last_msg.image_url is None:
            reply = Message(text="No image in the last message", source=self.type)
            message.chat_history.append(reply)
            await self.publish_message(
                message,
                DefaultTopicId(type=self._group_manager_topic),
            )
        
        userMessage = UserMessage([last_msg.text, Image.from_base64(last_msg.image_url)])
        chat_history.append(userMessage)
        completion = await self._model_client.complete_chat(chat_history)
        response = completion.messages[-1]
        reply = Message(text=response.text, source=self.type)
        message.chat_history.append(reply)
        await self.publish_message(
            message,
            DefaultTopicId(type=self._group_manager_topic),
        )

In [5]:
class WeatherAssistant(RoutedAgent):
    def __init__(self,
                 model_client: ChatCompletionClient,
                 system_message: str,
                 group_manager_topic: str) -> None:
        super().__init__(description=system_message)
        self._system_message = system_message
        self._group_manager_topic = group_manager_topic
        self._model_client = model_client
        self._get_weather_tool = FunctionTool(self.get_weather, description="get weather from a city and state")
        self._tools = [self._get_weather_tool]

    # get weather tool
    async def get_weather(self, city: str, state: str) -> str: # is the return type matter here?
    # get the weather for a city and state
        print(f"Getting the weather for {city}, {state}")
        return f"The weather in {city}, {state} is sunny."
    
    @message_handler
    async def handle_request_to_speak(self, message: Conversation, ctx: MessageContext) -> None:
        chat_history: List[LLMMessage] = [SystemMessage(content=self._system_message)]
        last_msg = message.chat_history[-1]
        if last_msg.image_url is not None:
            reply = Message(text="can't recognize image in the last message", source=self.type)
            message.chat_history.append(reply)
            await self.publish_message(
                message,
                DefaultTopicId(type=self._group_manager_topic),
            )
        chat_history.append(UserMessage(content=last_msg.text, source="user"))
        completion = await self._model_client.create(
            chat_history,
            tools=self._tools,
            extra_create_args={"tool_choice": "required"},
            cancellation_token=ctx.cancellation_token,
        )
        print(f"### {self.type}: \n{completion.content}")
        if not isinstance(completion.content, list):
            reply = Message(text="Error in tool call", source=self.type)
            print(f"### {self.type}: \n{reply.text}")
            message.chat_history.append(reply)
            await self.publish_message(
                message,
                DefaultTopicId(type=self._group_manager_topic),
            )

            return
        # run tool call
        for tool_call in completion.content:
            arguments = tool_call.arguments
            arguments = json.loads(tool_call.arguments)
            result = await self._get_weather_tool.run_json(arguments, ctx.cancellation_token)
            reply = Message(text=self._get_weather_tool.return_value_as_string(result), source=self.type)
            print(f"### {self.type}: \n{reply.text}")
            message.chat_history.append(reply)
            await self.publish_message(
                message,
                DefaultTopicId(type=self._group_manager_topic),
            )


In [6]:
class Assistant(RoutedAgent):
    def __init__(self,
                 model_client: ChatCompletionClient,
                 system_message: str,
                 group_manager_topic: str) -> None:
        super().__init__("An assistant agent.")
        self._model_client = model_client
        self._group_manager_topic = group_manager_topic
        self._system_message = system_message

    @message_handler
    async def handle_message(self, message: Conversation, ctx: MessageContext) -> None:
        chat_history: List[LLMMessage] = [SystemMessage(content=self._system_message)]
        for msg in message.chat_history:
            if msg.source != self.type:
                chat_history.append(UserMessage(content=msg.text, source=msg.source))
            else:
                chat_history.append(AssistantMessage(content=msg.text, source="assistant"))

        result = await self._model_client.create(chat_history)
        print(f"\n{'-'*80}\nAssistant:\n{result.content}")
        message.chat_history.append(Message(text=result.content, source=self.type))
        await self.publish_message(message, DefaultTopicId(type=self._group_manager_topic))  # type: ignore

In [7]:
class GroupChatManager(RoutedAgent):
    def __init__(self,
                    model_client: ChatCompletionClient,
                    weather_agent: str,
                    weather_agent_description: str,
                    image_agent: str,
                    image_agent_description: str,
                    user_agent: str,
                    user_agent_description: str,
                    assistant: str,
                    assistant_description: str) -> None:
        super().__init__("Group Chat Manager")
        self._model_client = model_client
        self._weather_agent = weather_agent
        self._weather_agent_description = weather_agent_description
        self._image_agent = image_agent
        self._image_agent_description = image_agent_description
        self._user_agent = user_agent
        self._user_agent_description = user_agent_description
        self._assistant = assistant
        self._assistant_description = assistant_description

    @message_handler
    async def handle_message(self, message: Conversation, ctx: MessageContext) -> None:
        # Format message history.
        messages: List[str] = []
        for msg in message.chat_history:
            messages.append(f"{msg.source}: {msg.text}")
        history = "\n".join(messages)
        # Format roles.
        roles = "\n".join(
            [
                f"{topic_type}: {description}".strip()
                for topic_type, description in zip(
                    [self._weather_agent, self._image_agent, self._user_agent, self._assistant],
                    [
                        self._weather_agent_description,
                        self._image_agent_description,
                        self._user_agent_description,
                        self._assistant_description,
                    ],
                )
            ]
        )
        selector_prompt = """You are in a role play game. The following roles are available:
{roles}.
Read the following conversation. Then select the next role from {participants} to play. Only return the role.

{history}

Read the above conversation. Then select the next role from {participants} to play. Only return the role.
"""
        available_roles = [
            self._weather_agent,
            self._image_agent,
            self._user_agent,
            self._assistant,
        ]
        system_message = SystemMessage(
            selector_prompt.format(
                roles=roles,
                history=history,
                participants=str(
                    [
                        topic_type
                        for topic_type in available_roles
                    ]
                ),
            )
        )

        completion = await self._model_client.create([system_message])
        assert isinstance(completion.content, str)
        for topic_type in available_roles:
            if topic_type.lower() in completion.content.lower():
                print(f"### {self.type}: \nSelected role: {topic_type}")
                await self.publish_message(message, DefaultTopicId(type=topic_type))
                return
        raise ValueError(f"Invalid role selected: {completion.content}")

In [10]:
runtime = SingleThreadedAgentRuntime()
gpt_4o_mini = OpenAIChatCompletionClient(
            api_key=os.environ.get("OPENAI_API_KEY"),
            model="gpt-4o-mini",
        )
gpt_4o = OpenAIChatCompletionClient(
            api_key=os.environ.get("OPENAI_API_KEY"),
            model="gpt-4o",
        )
user_agent_description = "A user agent that generates messages for the assistant agent."
user_agent = await UserAgent.register(
    runtime,
    type="user",
    factory=lambda: UserAgent(
        description=user_agent_description,
        group_manager_topic="admin",
    ),
)

image_agent_description = "An agent that processes images."
image_agent = await ImageAssistant.register(
    runtime,
    type="image",
    factory=lambda: ImageAssistant(
        model_client=gpt_4o_mini,
        system_message="You are an AI assistant that can process images.",
        group_manager_topic="admin",
    ),
)

weather_agent_description = "An agent that provides weather information."
weather_agent = await WeatherAssistant.register(
    runtime,
    type="weather",
    factory=lambda: WeatherAssistant(
        model_client=gpt_4o_mini,
        system_message="You are a helpful AI assistant that can provide weather information.",
        group_manager_topic="admin",
    ),
)

assistant_description = "An assistant agent that can help with various tasks."
assistant = await Assistant.register(
    runtime,
    type="assistant",
    factory=lambda: Assistant(model_client=gpt_4o_mini, system_message="You are a helpful AI assistant", group_manager_topic="admin"),
)

group_manager_description = "A group chat manager that routes messages to the appropriate agent."
group_manager = await GroupChatManager.register(
    runtime,
    type="admin",
    factory=lambda: GroupChatManager(
        model_client=gpt_4o,
        weather_agent=weather_agent.type,
        weather_agent_description=weather_agent_description,
        image_agent=image_agent.type,
        image_agent_description=image_agent_description,
        user_agent=user_agent.type,
        user_agent_description=user_agent_description,
        assistant=assistant.type,
        assistant_description=assistant_description,
    ),
)

await runtime.add_subscription(
    TypeSubscription("user", user_agent.type))
await runtime.add_subscription(
    TypeSubscription("assistant", assistant.type))
await runtime.add_subscription(
    TypeSubscription("image", image_agent.type))
await runtime.add_subscription(
    TypeSubscription("weather", weather_agent.type))
await runtime.add_subscription(
    TypeSubscription("admin", group_manager.type))

runtime.start()
session_id = str(uuid.uuid4())
msg = Conversation(chat_history=[])
await runtime.publish_message(
    msg,
    TopicId(type="user", source=session_id),
)

await runtime.stop_when_idle()

### User: 
Make a joke about newyork current weather


C:\Users\xiaoyuz\AppData\Local\Temp\ipykernel_33692\2242232903.py:72: UserWarning: Resolved model mismatch: gpt-4o-2024-05-13 != gpt-4o-2024-08-06. Model mapping may be incorrect.
  completion = await self._model_client.create([system_message])


### admin: 
Selected role: weather
### weather: 
[FunctionCall(id='call_Z8cke9Mcag1SFGHTyAW0BCOR', arguments='{"city":"New York","state":"NY"}', name='get_weather')]
Getting the weather for New York, NY
### weather: 
The weather in New York, NY is sunny.
### admin: 
Selected role: assistant

--------------------------------------------------------------------------------
Assistant:
Why did the sun go to school in New York?

Because it wanted to get a little "brighter" with all that sunny weather! ☀️
### admin: 
Selected role: user
### User: 
Make another one about seattle current weather
### admin: 
Selected role: weather
### weather: 
[FunctionCall(id='call_yYJuVrWjAB617SUiMNnqHPvW', arguments='{"city":"Seattle","state":"WA"}', name='get_weather')]
Getting the weather for Seattle, WA
### weather: 
The weather in Seattle, WA is sunny.
### admin: 
Selected role: assistant

--------------------------------------------------------------------------------
Assistant:
Why did the coffee file